In [18]:
import pandas as pd
df = pd.read_csv("dataset-loslagos-8months.csv", 
                 sep=";", 
                 engine='python')
df.columns = ["date", "media_outlet", "url", "title", "text"]
df["content"] = ""
df.content = df.title + " " + df.text
print("Largo del dataset: ", len(df))
df.head()

Largo del dataset:  29598


,date,media_outlet,url,title,text,content
0,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/reconocen-a-g...,Reconocen a guardaparques de la Región de Los ...,Distintos protagonistas de los parques naciona...,Reconocen a guardaparques de la Región de Los ...
1,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/con-nuevos-ma...,Con nuevos materiales comienza plan piloto en ...,Centro de negocios Sercotec coordina acuerdos ...,Con nuevos materiales comienza plan piloto en ...
2,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/centro-de-sal...,Centro de Salud Familiar CESFAM Puerto Varas i...,Las horas se solicitan en el SOME o bien a tra...,Centro de Salud Familiar CESFAM Puerto Varas i...
3,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/alcalde-tomas...,Alcalde Tomás Gárate presidió por primera vez ...,Los y las consejeras destacaron el hecho de vo...,Alcalde Tomás Gárate presidió por primera vez ...
4,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/galeria-de-ar...,Galería de Arte Machacoya realizará remate de ...,"Hoy viernes a las 18:30 horas, en Machacoya At...",Galería de Arte Machacoya realizará remate de ...


<img src="img/los_lagos.png" />

Fuente: [wikipedia](https://es.wikipedia.org/wiki/Portal:Regi%C3%B3n_de_Los_Lagos/Comunas_de_Los_Lagos)

In [19]:
comunas = ["ancud", "castro", "chonchi", "curaco de vélez", "dalcahue", 
           "puqueldón", "queilén", "quemchi", "quellón", "quinchao"
           "calbuco", "cochamó", "fresia", "frutillar", "llanquihue"
           "los muermos", "maullin", "puerto montt", "puerto varas", "osorno"
           "puerto octay", "purranque", "puyehue", "rio negro", "san juan de la costa",
           "san pablo", "chaitén", "futaleufú", "hualaihué", "palena"]

In [46]:
from tqdm.notebook import tqdm_notebook

df['comuna'] = ""
for index, row in tqdm_notebook(df.iterrows(), desc="buscando comunas en content"):
    found = [comuna for comuna in comunas if(comuna in str(row["content"]).lower())]
    if(len(found)!= 0): df.at[index, 'comuna'] = found


buscando comunas en content: 0it [00:00, ?it/s]

In [48]:
df['comuna'].value_counts()[:5]

                  19388
[puerto montt]     2545
[castro]           1212
[puerto varas]      726
[ancud]             570
Name: comuna, dtype: int64